# languages detector ML project

## import libraries

In [1]:
import nltk
from nltk.tokenize import sent_tokenize
import random
from nltk.collections import defaultdict
from nltk.tokenize import WordPunctTokenizer
#from hazm import *
import pandas as pd
tokenizer=WordPunctTokenizer()

nltk.download("punkt")

[nltk_data] Downloading package punkt to /Users/ngn/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

## class Language

In [11]:
class Language:
    
    def __init__(self,name):
        self.name=name
    
    def load_dataset(self,file):
        #"/Users/ngn/Desktop/english.txt"
        self.dataset=open(file,encoding="utf-8").read()
        return 'true'
    
    def seprate_train_test_set(self):
        self.sen=sent_tokenize(self.dataset)
        random.shuffle(self.sen)
        cut=int((4/5)*len(self.sen))
        self.train=self.sen[:cut]
        self.test=self.sen[cut:]
        return self.train,self.test
    
    def set_stop_word(self,stp):
        self.stp=stp
        
        
    def remove_1_2_chars(self):
        for word in self.bag_of_word:
            if len(word)<3:
                self.bag_of_word.remove(word)  
        
        
    def remove_special_chars(self,text):
        chars=['0','1','2','3','4','5','6','7','8','9','"',"'",'=','@','&','%','.',',',':','\\','$','^','<','>','!','?','{','}',';','\n','\t','(',')','[',']','/','*','+','#','\u200c','\ufeff','-','_','|','۱','۲','۳','۴','۵','۶','۷','۸','۹','۰','،']
        for item in chars:
            text=text.replace(item,"")
        return text
    
    def remove_stopwords(self):
        self.bag_of_word=[w for w in self.bag_of_word if w not in self.stp]
    
    def preproccessing(self):
        tokenizer=WordPunctTokenizer()
        text=''.join(self.train)
        text=self.remove_special_chars(text)
        self.bag_of_word=tokenizer.tokenize(text)
        self.remove_stopwords()
        self.remove_1_2_chars()
        #few more proccess here
        return self.bag_of_word,len(self.bag_of_word)

    
    def defualt_value(self,v):
        return (1/(len(self.bag_of_word)+v))
    
    
    def create_features_dict(self,defualt_value_0):
        self.features_dict=defaultdict(defualt_value_0)#defualt_value_0 static in main benevis
        for item in self.bag_of_word:
            self.features_dict[item]+=1
            
    def probability_conditional_all(self,v):
        # this for calculate p(key|english boodan)
        for key,val in self.features_dict.items():
            self.features_dict[key]=(val+1)/(len(self.bag_of_word)+v)
            
    def probability_conditional(self,feat,v):
        # this for calculate p(feat|english boodan)
        return (self.features_dict[feat]+1)/(len(self.bag_of_word)+v)
    
    def train_len(self):
        return len(self.train)
    
    
    

## Main

### 1.functions

In [12]:

def probability_class(train_lens):
    p_class=[]
    sl=sum(train_lens)
    for item in train_lens:
        p_class.append(int(item)/sl)
    return p_class


def test_case(string,languages,prob_classes):
    tokens=tokenizer.tokenize(string)#comment this line
    for feat in tokens:#tokens->string
        for i in range(len(languages)):
            prob_classes[i] *= languages[i].probability_conditional(feat,len(features))
            if prob_classes.count(0.0)==len(languages)-1:
                return prob_classes
    return prob_classes


def defualt_value_0():
    return 0


#testing functions

def make_test_list(languages):
    test_dict=dict()
    for lan in languages:
        for item in lan.test:
            test_dict.update({item: lan.name})
    return test_dict

def precesion(tp,fp):
    return tp/(tp+fp)

def test_checking(inputs):
    outputs=[]
    tp=0
    fp=0
    for item in inputs:
        p_test=test_case(item,languages,pc.copy())
        index=p_test.index(max(p_test))
        res=languages[index].name
        outputs.append(res)
        if res==inputs[item]:
            tp+=1
        else:
            fp+=1
    return outputs,tp,fp


def test_validation(inputs):
    for item in inputs:
        p_test=test_case(item[0],languages,pc.copy())
        index=p_test.index(max(p_test))
        res=languages[index].name
        item.append(languages[index].name)
    return inputs


### 2.training

#### 2-1.loading dataset and languages

###### set stop words

In [13]:
stp_arabic=['إذ', 'إذا', 'إذما', 'إذن', 'أف', 'أقل', 'أكثر', 'ألا', 'إلا', 'التي', 'الذي', 'الذين', 'اللاتي', 'اللائي', 'اللتان', 'اللتيا', 'اللتين', 'اللذان', 'اللذين', 'اللواتي', 'إلى', 'إليك', 'إليكم', 'إليكما', 'إليكن', 'أم', 'أما', 'أما', 'إما', 'أن', 'إن', 'إنا', 'أنا', 'أنت', 'أنتم', 'أنتما', 'أنتن', 'إنما', 'إنه', 'أنى', 'أنى', 'آه', 'آها', 'أو', 'أولاء', 'أولئك', 'أوه', 'آي', 'أي', 'أيها', 'إي', 'أين', 'أين', 'أينما', 'إيه', 'بخ', 'بس', 'بعد', 'بعض', 'بك', 'بكم', 'بكم', 'بكما', 'بكن', 'بل', 'بلى', 'بما', 'بماذا', 'بمن', 'بنا', 'به', 'بها', 'بهم', 'بهما', 'بهن', 'بي', 'بين', 'بيد', 'تلك', 'تلكم', 'تلكما', 'ته', 'تي', 'تين', 'تينك', 'ثم', 'ثمة', 'حاشا', 'حبذا', 'حتى', 'حيث', 'حيثما', 'حين', 'خلا', 'دون', 'ذا', 'ذات', 'ذاك', 'ذان', 'ذانك', 'ذلك', 'ذلكم', 'ذلكما', 'ذلكن', 'ذه', 'ذو', 'ذوا', 'ذواتا', 'ذواتي', 'ذي', 'ذين', 'ذينك', 'ريث', 'سوف', 'سوى', 'شتان', 'عدا', 'عسى', 'عل', 'على', 'عليك', 'عليه', 'عما', 'عن', 'عند', 'غير', 'فإذا', 'فإن', 'فلا', 'فمن', 'في', 'فيم', 'فيما', 'فيه', 'فيها', 'قد', 'كأن', 'كأنما', 'كأي', 'كأين', 'كذا', 'كذلك', 'كل', 'كلا', 'كلاهما', 'كلتا', 'كلما', 'كليكما', 'كليهما', 'كم', 'كم', 'كما', 'كي', 'كيت', 'كيف', 'كيفما', 'لا', 'لاسيما', 'لدى', 'لست', 'لستم', 'لستما', 'لستن', 'لسن', 'لسنا', 'لعل', 'لك', 'لكم', 'لكما', 'لكن', 'لكنما', 'لكي', 'لكيلا', 'لم', 'لما', 'لن', 'لنا', 'له', 'لها', 'لهم', 'لهما', 'لهن', 'لو', 'لولا', 'لوما', 'لي', 'لئن', 'ليت', 'ليس', 'ليسا', 'ليست', 'ليستا', 'ليسوا', 'ما', 'ماذا', 'متى', 'مذ', 'مع', 'مما', 'ممن', 'من', 'منه', 'منها', 'منذ', 'مه', 'مهما', 'نحن', 'نحو', 'نعم', 'ها', 'هاتان', 'هاته', 'هاتي', 'هاتين', 'هاك', 'هاهنا', 'هذا', 'هذان', 'هذه', 'هذي', 'هذين', 'هكذا', 'هل', 'هلا', 'هم', 'هما', 'هن', 'هنا', 'هناك', 'هنالك', 'هو', 'هؤلاء', 'هي', 'هيا', 'هيت', 'هيهات', 'والذي', 'والذين', 'وإذ', 'وإذا', 'وإن', 'ولا', 'ولكن', 'ولو', 'وما', 'ومن', 'وهو', 'يا']
stp_english=['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
stp_german=['aber', 'alle', 'allem', 'allen', 'aller', 'alles', 'als', 'also', 'am', 'an', 'ander', 'andere', 'anderem', 'anderen', 'anderer', 'anderes', 'anderm', 'andern', 'anderr', 'anders', 'auch', 'auf', 'aus', 'bei', 'bin', 'bis', 'bist', 'da', 'damit', 'dann', 'der', 'den', 'des', 'dem', 'die', 'das', 'dass', 'daß', 'derselbe', 'derselben', 'denselben', 'desselben', 'demselben', 'dieselbe', 'dieselben', 'dasselbe', 'dazu', 'dein', 'deine', 'deinem', 'deinen', 'deiner', 'deines', 'denn', 'derer', 'dessen', 'dich', 'dir', 'du', 'dies', 'diese', 'diesem', 'diesen', 'dieser', 'dieses', 'doch', 'dort', 'durch', 'ein', 'eine', 'einem', 'einen', 'einer', 'eines', 'einig', 'einige', 'einigem', 'einigen', 'einiger', 'einiges', 'einmal', 'er', 'ihn', 'ihm', 'es', 'etwas', 'euer', 'eure', 'eurem', 'euren', 'eurer', 'eures', 'für', 'gegen', 'gewesen', 'hab', 'habe', 'haben', 'hat', 'hatte', 'hatten', 'hier', 'hin', 'hinter', 'ich', 'mich', 'mir', 'ihr', 'ihre', 'ihrem', 'ihren', 'ihrer', 'ihres', 'euch', 'im', 'in', 'indem', 'ins', 'ist', 'jede', 'jedem', 'jeden', 'jeder', 'jedes', 'jene', 'jenem', 'jenen', 'jener', 'jenes', 'jetzt', 'kann', 'kein', 'keine', 'keinem', 'keinen', 'keiner', 'keines', 'können', 'könnte', 'machen', 'man', 'manche', 'manchem', 'manchen', 'mancher', 'manches', 'mein', 'meine', 'meinem', 'meinen', 'meiner', 'meines', 'mit', 'muss', 'musste', 'nach', 'nicht', 'nichts', 'noch', 'nun', 'nur', 'ob', 'oder', 'ohne', 'sehr', 'sein', 'seine', 'seinem', 'seinen', 'seiner', 'seines', 'selbst', 'sich', 'sie', 'ihnen', 'sind', 'so', 'solche', 'solchem', 'solchen', 'solcher', 'solches', 'soll', 'sollte', 'sondern', 'sonst', 'über', 'um', 'und', 'uns', 'unsere', 'unserem', 'unseren', 'unser', 'unseres', 'unter', 'viel', 'vom', 'von', 'vor', 'während', 'war', 'waren', 'warst', 'was', 'weg', 'weil', 'weiter', 'welche', 'welchem', 'welchen', 'welcher', 'welches', 'wenn', 'werde', 'werden', 'wie', 'wieder', 'will', 'wir', 'wird', 'wirst', 'wo', 'wollen', 'wollte', 'würde', 'würden', 'zu', 'zum', 'zur', 'zwar', 'zwischen']
stp_french=['au', 'aux', 'avec', 'ce', 'ces', 'dans', 'de', 'des', 'du', 'elle', 'en', 'et', 'eux', 'il', 'ils', 'je', 'la', 'le', 'les', 'leur', 'lui', 'ma', 'mais', 'me', 'même', 'mes', 'moi', 'mon', 'ne', 'nos', 'notre', 'nous', 'on', 'ou', 'par', 'pas', 'pour', 'qu', 'que', 'qui', 'sa', 'se', 'ses', 'son', 'sur', 'ta', 'te', 'tes', 'toi', 'ton', 'tu', 'un', 'une', 'vos', 'votre', 'vous', 'c', 'd', 'j', 'l', 'à', 'm', 'n', 's', 't', 'y', 'été', 'étée', 'étées', 'étés', 'étant', 'étante', 'étants', 'étantes', 'suis', 'es', 'est', 'sommes', 'êtes', 'sont', 'serai', 'seras', 'sera', 'serons', 'serez', 'seront', 'serais', 'serait', 'serions', 'seriez', 'seraient', 'étais', 'était', 'étions', 'étiez', 'étaient', 'fus', 'fut', 'fûmes', 'fûtes', 'furent', 'sois', 'soit', 'soyons', 'soyez', 'soient', 'fusse', 'fusses', 'fût', 'fussions', 'fussiez', 'fussent', 'ayant', 'ayante', 'ayantes', 'ayants', 'eu', 'eue', 'eues', 'eus', 'ai', 'as', 'avons', 'avez', 'ont', 'aurai', 'auras', 'aura', 'aurons', 'aurez', 'auront', 'aurais', 'aurait', 'aurions', 'auriez', 'auraient', 'avais', 'avait', 'avions', 'aviez', 'avaient', 'eut', 'eûmes', 'eûtes', 'eurent', 'aie', 'aies', 'ait', 'ayons', 'ayez', 'aient', 'eusse', 'eusses', 'eût', 'eussions', 'eussiez', 'eussent']
stp_persian=['و','در','به','از','که','این','را','با','است','برای','آن','یک','خود','تا','کرد','بر','هم','نیز','گفت','وی','شد','دارد','ما','اما','یا','شده','باید','هر','آنها','بود','او','یگر','دو','مورد','شود','کند','وجود','بین','پیش','شده_است','پس','نظر','اگر','همه','یکی','حال','هستند','من','کنند','نیست','باشد','چه','بی','می','بخش','همین','افزود','هایی','دارند','راه','همچنین','روی','داد','بیشتر','بسیار','سه','داشت','چند','سوی','تنها','هیچ','میان','اینکه','شدن','بعد','جدید','ولی','حتی','کردن','برخی','کردند','اول','نه','کرده_است','نسبت','بیش','شما','چنین','طور','افراد','تمام','درباره','بار','بسیاری','کرده','چون','ندارد','دوم','بزرگ','طی','حدود','همان','بدون','البته','آنان','دیگری','خواهد_شد','کنیم','قابل','یعنی','رشد','وارد','کل','ویژه','قبل','براساس','نیاز','گذاری','هنوز','لازم','سازی','بوده_است','چرا','وقتی','گرفت','کم','جای','حالی','تغییر','پیدا','اکنون','تحت','باعث','مدت','فقط','زیادی','تعداد','آیا','بیان','رو','شدند','عدم','کرده_اند','بودن','نوع','بلکه','جاری','دهد','برابر','مهم','بوده','اخیر','مربوط','امر','زیر','گیری','شاید','خصوص','آقای','اثر','کننده','بودند','فکر','کنار','اولین','سوم','سایر','کنید','ضمن','مانند','باز','ممکن','حل','دارای','پی','مثل','اجرا','دور','منظور','کسی','موجب','طول','امکان','آنچه','تعیین','گفته','شوند','جمع','خیلی','علاوه','گونه','تاکنون','رسید','ساله','گرفته','شده_اند','علت','چهار','داشته_باشد','خواهد_بود','طرف','تهیه','تبدیل','مناسب','زیرا','مشخص','نزدیک','جریان','روند','بنابراین','یافت','نخستین','بالا','پنج','ریزی','عالی','چیزی','نخست','بیشتری','ترتیب','شده_بود','خاص','خوبی','خوب','شروع','فرد','کامل','غیر','دهند','آخرین','دادن','جدی','بهترین','شامل','گیرد','بخشی','باشند','تمامی','بهتر','داده_است','حد','نبود','کسانی','داریم','علیه','دانست','ناشی','داشتند','دهه','ایشان','آنجا','گرفته_است','دچار','لحاظ','آنکه','داده','بعضی','هستیم','اند','برداری','نباید','نشست','سهم','همیشه','آمد','اش','وگو','حداقل','طبق','جا','خواهد_کرد','نوعی','چگونه','رفت','هنگام','فوق','روش','ندارند','سعی','بندی','شمار','کلی','کافی','مواجه','همچنان','زیاد','سمت','کوچک','داشته_است','چیز','پشت','آورد','حالا','روبه','دادند','عهده','نیمه','جایی','دیگران','سی','بروز','یکدیگر','آمده_است','جز','کنم','سپس','کنندگان','خودش','همواره','یافته','شان','صرف','رسیدن','چهارم','یابد','متر','ساز','داشته','کرده_بود','باره','نحوه','کردم','تو','شخصی','داشته_باشند','محسوب','پخش','کمی','متفاوت','سراسر','کاملا','داشتن','نظیر','آمده','گروهی','فردی','ع','همچون','خطر','خویش','کدام','دسته','سبب','عین','آوری','متاسفانه','بیرون','دار','ابتدا','شش','افرادی','سالهای','درون','نیستند','یافته_است','پر','خاطرنشان','گاه','جمعی','اغلب','دوباره','لذا','زاده','گردد','اینجا']

In [14]:
english=Language("English")
german=Language("German")
french=Language("French")
persian=Language("Persian")
arabic=Language("Arabic")

english.load_dataset("/Users/ngn/Desktop/english.txt")
german.load_dataset("/Users/ngn/Desktop/german.txt")
french.load_dataset("/Users/ngn/Desktop/france.txt")
persian.load_dataset("/Users/ngn/Desktop/farsi.txt")
arabic.load_dataset("/Users/ngn/Desktop/arabic.txt")

english.set_stop_word(stp_english)
french.set_stop_word(stp_french)
german.set_stop_word(stp_german)
persian.set_stop_word(stp_persian)
arabic.set_stop_word(stp_arabic)


#### 2-2.initialize dataset and languages

In [15]:
languages=[english,german,french,persian,arabic]
train_lens=[]
features=set()
for l in languages:
    l.seprate_train_test_set()
    l.preproccessing()
    l.create_features_dict(defualt_value_0)
    train_lens.append(int(len(l.train)))
    features=features.union(set(l.features_dict))
    
pc=probability_class(train_lens)# probability of each class in a list

### 5.run

In [16]:
#for a list of tess cases

test_list=make_test_list(languages)
outputs,tp,fp=test_checking(test_list)

#precesion(tp,fp)

In [17]:
#for 1 test case

test_string="hello language detector!"
pt=test_case(test_string,languages,pc.copy())# probability of test case in a list

index=pt.index(max(pt))
detected_language=languages[index].name

#### validation

In [18]:
document=pd.read_csv('/Users/ngn/Desktop/test.csv', header=0)
validation=document.values.tolist()

resault_list=test_validation(validation)

columns = ["Id","Category"]
data = [i[0:2] for i in resault_list] 
pd2 = pd.DataFrame(data=data, columns = columns)
pd2.to_csv("sample1 - task1.csv",index=False)